In [1]:
import pandas as pd
import ftfy

In [2]:
# Load the CSV file with low_memory=False and without specifying dtypes initially
df = pd.read_csv('raw_movie_data.csv', low_memory=False)

# Convert relevant columns to numeric, coercing errors into NaN
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')
df['Budget'] = pd.to_numeric(df['Budget'], errors='coerce')
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
df['Ratings'] = pd.to_numeric(df['Ratings'], errors='coerce')
df['Vote Count'] = pd.to_numeric(df['Vote Count'], errors='coerce')
df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce')
df['Movie ID'] = pd.to_numeric(df['Movie ID'], errors='coerce')

# Convert the specified columns to integers (removing decimal places)
df['Year'] = df['Year'].astype('Int64')
df['Duration'] = df['Duration'].astype('Int64')
df['Budget'] = df['Budget'].astype('Int64')
df['Revenue'] = df['Revenue'].astype('Int64')
df['Vote Count'] = df['Vote Count'].astype('Int64')
df['Movie ID'] = df['Movie ID'].astype('Int64')

# Convert all object columns to string type (handling NaN as <NA> in string columns)
df['Title'] = df['Title'].astype('string')
df['Director'] = df['Director'].astype('string')
df['Producer'] = df['Producer'].astype('string')
df['Genres'] = df['Genres'].astype('string')
df['Summary'] = df['Summary'].astype('string')
df['Content Rating'] = df['Content Rating'].astype('string')
df['Original Language'] = df['Original Language'].astype('string')
df['Production Companies'] = df['Production Companies'].astype('string')
df['Production Countries'] = df['Production Countries'].astype('string')
df['Spoken Languages'] = df['Spoken Languages'].astype('string')
df['Tagline'] = df['Tagline'].astype('string')

# Drop the 'Adult' column from the DataFrame
df = df.drop(columns=['Adult'])

In [3]:
# Disable scientific notation for floats
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df.dtypes

Title                   string[python]
Year                             Int64
Director                string[python]
Producer                string[python]
Genres                  string[python]
Summary                 string[python]
Duration                         Int64
Budget                           Int64
Revenue                          Int64
Ratings                        float64
Vote Count                       Int64
Popularity                     float64
Content Rating          string[python]
Original Language       string[python]
Production Companies    string[python]
Production Countries    string[python]
Spoken Languages        string[python]
Tagline                 string[python]
Movie ID                         Int64
dtype: object

In [5]:
# Remove movies that have no summary (NaN or empty strings)
df = df.dropna(subset=['Summary'])

In [6]:
# Get number of rows prior to genre cleaning
pre_genre_clean = len(df)

# Remove films with no genres (NaN or empty)
df = df[df['Genres'].notna() & (df['Genres'] != '')]

# Get number of rows after genre cleaning
post_genre_clean = len(df)

# Check how many movies were removed
removed_no_genre_count = pre_genre_clean - post_genre_clean
print(f"Removed {removed_no_genre_count} films with no genres.")

Removed 1511 films with no genres.


In [7]:
def is_ascii(s):
    """Check if the string is ASCII."""
    return all(ord(c) < 128 for c in s)

# Function to filter out non-ASCII items in a comma-separated list
def remove_non_ascii_languages(df, column_name):
    def filter_ascii_list(items):
        # Split the list and filter each item
        return ', '.join([item for item in items.split(',') if is_ascii(item.strip())])

    # Apply the function to each row in the specified column
    df[column_name] = df[column_name].apply(lambda x: filter_ascii_list(str(x)) if isinstance(x, str) else x)
    return df

# Function to remove empty strings from comma-separated lists
def remove_empty_strings(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: ', '.join(filter(lambda s: s.strip() != '', x.split(','))) if isinstance(x, str) else x)
    return df

# Apply to relevant columns
df = remove_non_ascii_languages(df, 'Original Language')
df = remove_non_ascii_languages(df, 'Spoken Languages')
df = remove_non_ascii_languages(df, 'Production Companies')
df = remove_non_ascii_languages(df, 'Production Countries')

# Remove empty strings from the language lists
df = remove_empty_strings(df, 'Original Language')
df = remove_empty_strings(df, 'Spoken Languages')
df = remove_empty_strings(df, 'Production Companies')
df = remove_empty_strings(df, 'Production Countries')

In [8]:
# Replace NaN values with an empty string for the relevant columns
columns_to_process = ['Genres','Original Language', 'Spoken Languages', 'Production Companies', 'Production Countries']
df[columns_to_process] = df[columns_to_process].fillna("")

# Split each of these columns by commas, creating lists from the strings
for col in columns_to_process:
    df[col] = df[col].apply(lambda x: [item.strip() for item in x.split(',')] if x else [])

In [9]:
# Define the list of genres that represent stand-up comedy
standup_genres = ['Comedy', 'Documentary']

# Function to check if a movie is stand-up based on its genres
def is_standup(genres):
    # Sort both lists so that order doesn't affect the comparison
    return sorted(genres) == sorted(standup_genres)

# Get number of rows prior to stand-up removal
pre_genre_clean = len(df)

# Apply the filter to remove stand-up comedy movies
df = df[~df['Genres'].apply(is_standup)]

# Get number of rows after stand-up removal
post_genre_clean = len(df)

# Check how many movies were removed
removed_standup_count = pre_genre_clean - post_genre_clean
print(f"Removed {removed_standup_count} thought as stand-up comedy.")

Removed 136 thought as stand-up comedy.


In [10]:
# Replace NaN values in string columns with empty strings - some movies don't have production companies
string_columns = df.select_dtypes(include=['object', 'string']).columns
df[string_columns] = df[string_columns].fillna("")

In [11]:
# Get number of rows prior to director or producer removal
pre_nan_clean = len(df)

# Remove movies with no Director or Producer (including empty strings)
df = df[(df['Director'].notna()) & (df['Director'] != '') & 
        (df['Producer'].notna()) & (df['Producer'] != '')]

# Get number of rows after director or producer removal
post_nan_clean = len(df)

# Check how many movies were removed
removed_dir_or_pro = pre_nan_clean - post_nan_clean
print(f"Removed {removed_dir_or_pro} as they had no director or producer.")

Removed 11407 as they had no director or producer.


In [12]:
# Doesn't appear all string fields were fixed during data aquisition, apply FTFY to all string columns in the DataFrame - some languages won't be fixed
string_columns = df.select_dtypes(include=['string']).columns

# Fix the text in each of the string columns
for col in string_columns:
    df[col] = df[col].apply(ftfy.fix_text)

In [13]:
# Get row count prior to removing duplicate movies
pre_dupe_remove = len(df)

# Remove duplicate movies based on 'Title' and 'Year' - have found many duplicate movies
df = df.drop_duplicates(subset=['Title', 'Year'], keep='first')  # keep='first' retains the first occurrence

# Get row count after removing duplicate movies
post_dupe_remove = len(df)

# Calculate duplicates removed
dupes_removed = pre_dupe_remove - post_dupe_remove

print(dupes_removed)

1299


In [14]:
# Specify the path for the new cleaned CSV file
cleaned_file_path = 'cleaned_movie_data.csv'

# Save the cleaned DataFrame to a new CSV file
df.to_csv(cleaned_file_path, index=False)